# NRS - Project Work - Group 1

### Libraries and Data import

In [219]:
import pandas as pd
import numpy as np

%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

#for sentiment analysis
from textblob import TextBlob

missing_values = ['','Undefined', 'Undefined->Undefined'] #define values to be identified as NA in the datasets 

df_crm = pd.read_csv('crm_model.csv', keep_default_na=False, na_values=missing_values) #import CRM_model
df_finance = pd.read_csv('finance_model.csv', keep_default_na=False, na_values=missing_values) #import finance_model
df_sales = pd.read_csv('sales_model.csv', keep_default_na=False, na_values=missing_values) #import sales_model
df_twitter = pd.read_csv('twitter_model.csv', keep_default_na=False, na_values=missing_values) #import twitter_model

### Joining Datasets

In [220]:
#column id renaming to join the datasets
df_crm = df_crm.rename(columns={'ID_CRM': 'id'}) 
df_finance = df_finance.rename(columns={'ID_FINANCE': 'id'})
df_sales = df_sales.rename(columns={'ID_SALES': 'id'})
df_twitter = df_twitter.rename(columns={'ID_SALES': 'id'})

By observing the structure of each id we find a pattern to extract a common unique id:

<img src="img/ID_schema.png"></img>

In [221]:
#df_crm unique_id extraction
df_crm.id = np.where(df_crm.Income_Level.str.len() == 2,\
                     df_crm.id.str.slice(start = 1, stop = -2),\
                     df_crm.id.str.slice(start = 1, stop = -1))

#df_finance unique_id extraction
df_finance.id = np.where(df_finance.Special_Pay.str.len() == 2,\
                         df_finance.id.str.slice(start = 2),\
                         df_finance.id.str.slice(start = 0))

#df_sales unique_id extraction
df_sales.id = np.where(df_sales.Program_Code.str.len() == 2,\
                       df_sales.id.str.slice(start = 2, stop = -1),\
                       df_sales.id.str.slice(start = 3, stop = -1))

#df_twitter unique_id extraction
df_twitter.id = df_twitter.id.str.extract(r'(\d+)', expand=False)

In [222]:
# create a single table to join the first 3 datasets
df_total = df_sales.merge(df_crm, on="id", how="left").merge(df_finance, on="id", how="left") #.merge(df_twitter, on="unique_id", how="left")

In [225]:
#to export in excel
#df_total.to_excel("total.xlsx")

### Consistency check on Data (TBC)

In [226]:
df_total['RPL_delta'] = pd.to_datetime(df_total['Latest_RPL']) - pd.to_datetime(df_total['Early_RPL'])
df_total[{'Latest_RPL','Early_RPL','RPL_delta'}].loc[df_total['RPL_delta'].dt.days < 0]

,Latest_RPL,RPL_delta,Early_RPL
0,04/10/2018,-31 days,05/11/2018
8,03/10/2018,-3 days,03/13/2018
12,10/26/2017,-13 days,11/08/2017
14,04/22/2018,-24 days,05/16/2018
19,01/28/2018,-1 days,01/29/2018
...,...,...,...
3905,07/22/2018,-27 days,08/18/2018
4002,03/31/2018,-20 days,04/20/2018
4074,03/08/2018,-22 days,03/30/2018
4118,04/23/2018,-7 days,04/30/2018


In [227]:
df_total['Days_check'] = pd.to_datetime(df_total['Return_Date']) - pd.to_datetime(df_total['Departure_Date'])
df_total[{'Departure_Date','Return_Date','Days', 'Days_check'}]

,Days_check,Departure_Date,Days,Return_Date
0,24 days,04/07/2019,24,05/01/2019
1,18 days,04/30/2019,18,05/18/2019
2,13 days,03/17/2019,13,03/30/2019
3,24 days,05/08/2019,24,06/01/2019
4,13 days,03/17/2019,13,03/30/2019
...,...,...,...,...
4148,18 days,04/04/2019,18,04/22/2019
4149,13 days,03/21/2019,13,04/03/2019
4150,20 days,04/16/2019,20,05/06/2019
4151,10 days,06/01/2019,10,06/11/2019


In [228]:
df_total['FPP_to_School_enrollment_check'] = df_total['FPP'] / df_total['Total_School_Enrollment']
df_total[{'FPP_to_School_enrollment_check', 'FPP_to_School_enrollment', 'FPP', 'Total_School_Enrollment'}]

,FPP_to_School_enrollment,FPP_to_School_enrollment_check,Total_School_Enrollment,FPP
0,"0,126429354314411",0.132653,196.0,26
1,"0,173405156631824",0.205882,102.0,21
2,"0,108985714542943",NaN,NaN,7
3,"0,142807017036153",0.164557,158.0,26
4,"0,108985714542943",0.127273,55.0,7
...,...,...,...,...
4148,"0,014665665548888",0.018634,644.0,12
4149,"0,01776427530478",0.014317,908.0,13
4150,"0,006920828413484",0.007233,553.0,4
4151,"0,009419789252674",0.007955,1257.0,10


In [229]:
df_total['Total_Pax_check'] = df_total['FPP'] + df_total['Total_Discount_Pax'] + df_total['Num_of_Non_FPP_PAX']
df_total[{'Total_Pax_check','Total_Pax','FPP','Total_Discount_Pax','Num_of_Non_FPP_PAX'}]

,Total_Pax_check,FPP,Total_Pax,Num_of_Non_FPP_PAX,Total_Discount_Pax
0,28.0,26,29,1.0,1
1,27.0,21,22,3.0,3
2,NaN,7,7,NaN,1
3,32.0,26,25,3.0,3
4,9.0,7,7,1.0,1
...,...,...,...,...,...
4148,14.0,12,11,1.0,1
4149,15.0,13,17,1.0,1
4150,8.0,4,6,2.0,2
4151,14.0,10,12,2.0,2


### Topic & Sentiment Analysis of tweets (TBC)

In [230]:
df_twitter['frequency'] = df_twitter['id'].map(df_twitter['id'].value_counts())

In [231]:
df_twitter['message'] = df_twitter.groupby('id')['text'].transform(lambda x: ','.join(x))
df_twitter_merged = df_twitter[{'id','frequency','message'}].drop_duplicates()
df_twitter_merged.reset_index(drop=True, inplace=True)

#pd.set_option('display.max_colwidth', 50)
df_twitter_merged

,frequency,id,message
0,3,1927,"Lastly, for a company that's trying to put th..."
1,3,4244,what about for us that cant get to the states...
2,3,108,an already pleasant flight from London to Cha...
3,3,4335,She could even see that I had tried to make t...
4,3,416,but seriously if my cats dead I'm going to be...
...,...,...,...
4143,3,2795,"Headphone jack not working on my flight., as ..."
4144,3,2383,:take note of this great example of actually...
4145,3,3570,Northern California coast http://t.co/nm4VNN...
4146,3,4426,"I am But Your customer service is Brutal., Ha..."


In [239]:
# sentiment analysis
sentence = "Generali is the best insurance company!" 
#sentence = df_twitter_merged[df_twitter_merged['id'] == '4426'].message.to_numpy()[0]
analysis = TextBlob(sentence).sentiment
analysis

Sentiment(polarity=1.0, subjectivity=0.3)